In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/Remi/

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1WJLnwO7mUn1QFbhFIJ6fMZjZWuCKf1ub/Remi


In [2]:
!pip install simpletransformers
!pip install datasets
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 29.3 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 
     |████████████████████████████████| 4.4 MB 44.2 MB/s 
     |████████████████████████████████| 362 kB 69.2 MB/s 
     |████████████████████████████████| 1.8 MB 41.9 MB/s 
     |████████████████████████████████| 9.1 MB 43.0 MB/s 
     |████████████████████████████████| 1.2 MB 51.3 MB/s 
     |████████████████████████████████| 101 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 181 kB 58.3 MB/s 
     |████████████████████████████████| 145 kB 61.8 MB/s 
     |████████████████████████████████| 63 kB 855 kB/s 
     |████████████████████████████████| 1.1 MB 71.3 MB/s 
     |████████████████████████████████| 140 kB 63.9 MB/s 
     |██████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from datasets import load_metric, load_dataset
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report, f1_score
import pandas as pd
import logging
from sklearn.utils import shuffle

!nvcc --version

cuda_available = torch.cuda.is_available()
print(torch.cuda.device_count())
print(cuda_available)

data = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-english/CT22_english_1A_checkworthy_train.tsv', sep='\t')
data_test = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-english/CT22_english_1A_checkworthy_dev_test.tsv', sep='\t')
data_dev = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-english/CT22_english_1A_checkworthy_dev.tsv', sep='\t')

data_nl = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-dutch/CT22_dutch_1A_checkworthy_train.tsv', sep='\t')
data_nl_test = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-dutch/CT22_dutch_1A_checkworthy_dev_test.tsv', sep='\t')
data_nl_dev = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-dutch/CT22_dutch_1A_checkworthy_dev.tsv', sep='\t')

data_off_test = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-english/test/CT22_english_1A_checkworthy_test_gold.tsv', sep='\t')

data_nl_off_test = pd.read_csv('clef2022-checkthat-lab-main/task1/data/subtasks-dutch/test/CT22_dutch_1A_checkworthy_test_gold.tsv', sep='\t')

del data["topic"]
del data["tweet_url"]
del data["tweet_id"]
data.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_test["topic"]
del data_test["tweet_url"]
del data_test["tweet_id"]
data_test.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_off_test["topic"]
del data_off_test["tweet_url"]
del data_off_test["tweet_id"]
data_off_test.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_dev["topic"]
del data_dev["tweet_url"]
del data_dev["tweet_id"]
data_dev.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

data['label'] = data['label'].astype(int)
data_test['label'] = data_test['label'].astype(int)
data_dev['label'] = data_dev['label'].astype(int)

del data_nl["topic"]
del data_nl["tweet_url"]
del data_nl["tweet_id"]
data_nl.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_nl_test["topic"]
del data_nl_test["tweet_url"]
del data_nl_test["tweet_id"]
data_nl_test.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_nl_off_test["topic"]
del data_nl_off_test["tweet_url"]
del data_nl_off_test["tweet_id"]
data_nl_off_test.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

del data_nl_dev["topic"]
del data_nl_dev["tweet_url"]
del data_nl_dev["tweet_id"]
data_nl_dev.rename(columns={"tweet_text":"text", "class_label":"label"} ,inplace=True)

data_nl['label'] = data_nl['label'].astype(int)
data_nl_test['label'] = data_nl_test['label'].astype(int)
data_nl_dev['label'] = data_nl_dev['label'].astype(int)


data = shuffle(data, random_state=111)
data_dev = shuffle(data_dev, random_state=111)
data_test = shuffle(data_test, random_state=111)

data_nl = shuffle(data_nl, random_state=111)
data_nl_dev = shuffle(data_nl_dev, random_state=111)
data_nl_test = shuffle(data_nl_test, random_state=111)

ones_subset_nl = data_nl.loc[data_nl["label"] == 1, :]
number_of_1s_nl = len(ones_subset_nl)
zero_subset_nl = data_nl.loc[data_nl["label"] == 0, :]
number_of_0s_nl = len(zero_subset_nl)
print("NL dataset: ", number_of_1s_nl)
print("NL dataset: ", number_of_0s_nl)

ones_subset = data.loc[data["label"] == 1, :]
number_of_1s = len(ones_subset)
zero_subset = data.loc[data["label"] == 0, :]
number_of_0s = len(zero_subset)
print("EN dataset: ", number_of_1s)
print("EN dataset: ", number_of_0s)

zero_subset = zero_subset.sample(number_of_0s_nl)
ones_subset = ones_subset.sample(number_of_1s_nl)

data_downsampled = pd.concat([ones_subset, zero_subset])
data_downsampled = shuffle(data_downsampled, random_state=111)

# unbalanced downsampled
ones_subset = data.loc[data["label"] == 1, :]
zero_subset = data.loc[data["label"] == 0, :]
number_of_1s = 194
number_of_0s = 729

zero_subset = zero_subset.sample(number_of_0s)
ones_subset = ones_subset.sample(number_of_1s)

data_down_unbalanced = pd.concat([ones_subset, zero_subset])
data_down_unbalanced = shuffle(data_down_unbalanced, random_state=111)

print(data_down_unbalanced)
print("EN unbalanced dataset: ", number_of_1s)
print("EN unbalanced dataset: ", number_of_0s)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1
True
NL dataset:  377
NL dataset:  546
EN dataset:  447
EN dataset:  1675
                                                   text  label
567   World-renowned immunologist/virologist who gre...      0
292   Instagram’s algorithm is suggesting posts that...      1
309   Over 12 million COVAX Vaccines have now reache...      0
1024  Netanyahu says that the vaccines are safe and ...      0
367   The UK's medicine regulator said on Thursday t...      0
...                                                 ...    ...
912   @DWUhlfelderLaw CVS is making appointments ava...      0
1667  ‘Covid-19 Vaccines Are Weapons of Mass Destruc...      1
1211  Jack is 87. His wife just turned 85 and lives ...      0
358   MD jurisdictions are now -- officially -- gett...      0
863   @funder 1) Hired an ho

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

frames = [data, data_nl]

frames_unbalanced = [data_down_unbalanced, data_nl]
data_unbalanced_en_nl = pd.concat(frames_unbalanced)
data_unbalanced_en_nl = shuffle(data_unbalanced_en_nl, random_state=111)

frames_dev = [data_dev, data_nl_dev]

data_dev_en_nl = pd.concat(frames_dev) 

data_en_nl = pd.concat(frames)

# Optional model configuration
model_args = ClassificationArgs(optimizer="Adafactor", train_batch_size=32, eval_batch_size=32, adafactor_relative_step=False, evaluate_during_training=True, evaluate_during_training_steps=15, max_seq_length=300, evaluate_during_training_verbose=True, adafactor_scale_parameter=False, adafactor_warmup_init=False, manual_seed=111, num_train_epochs=5, save_eval_checkpoints=False, save_optimizer_and_scheduler=False, save_steps=-1, overwrite_output_dir=True, wandb_project="visualization-demo")

# # Create a ClassificationModel
model = ClassificationModel(
    "roberta", "pdelobelle/robbert-v2-dutch-base", args=model_args, use_cuda=True, num_labels=2
)

# model = ClassificationModel(
#     "roberta", "final_models/adafactor/balancing_and_sampling/en-xlmroberta", use_cuda=True, args=model_args, num_labels=2
# )

# Train the model
model.train_model(data_nl, eval_df=data_nl_dev, f1=f1_score)

# Make predictions with the model
predictions, raw_outputs = model.predict(data_nl_test['text'].tolist())
bert_report = classification_report(data_nl_test['label'].tolist(), predictions, digits=4)
print("Dev test results:")
print(bert_report)

predictions, raw_outputs = model.predict(data_nl_dev['text'].tolist())
bert_report = classification_report(data_nl_dev['label'].tolist(), predictions, digits=4)
print("Dev results:")
print(bert_report)

predictions, raw_outputs = model.predict(data_nl_off_test['text'].tolist())
bert_report = classification_report(data_nl_off_test['label'].tolist(), predictions, digits=4)
print("Official test:")
print(bert_report)

predictions, raw_outputs = model.predict(data['text'].tolist())
bert_report = classification_report(data['label'].tolist(), predictions, digits=4)
print("Training data:")
print(bert_report)

Downloading:   0%|          | 0.00/660 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446M [00:00<?, ?B/s]

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


Official test:
              precision    recall  f1-score   support

           0     0.6005    0.7257    0.6572       350
           1     0.6049    0.4652    0.5259       316

    accuracy                         0.6021       666
   macro avg     0.6027    0.5955    0.5916       666
weighted avg     0.6026    0.6021    0.5949       666



  0%|          | 0/2122 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

Training data:
              precision    recall  f1-score   support

           0     0.8181    0.8513    0.8344      1675
           1     0.3430    0.2908    0.3148       447

    accuracy                         0.7333      2122
   macro avg     0.5806    0.5711    0.5746      2122
weighted avg     0.7180    0.7333    0.7249      2122

